In [1]:
# !pip install PyPDF2

In [2]:
pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 69.3 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.10.1 requires cubinlinker, which is not installed.
cudf 2

In [4]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 87.3 MB/s eta 0:00:00:00:0100:01


In [5]:
# import PyPDF2
# from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import fitz
import re


In [6]:
# def extract_text_from_pdf(path):
#     with open(path,'r') as file:
#         pdf_reader=PyPDF2.PdfReader(file)
#         text=""
#         for page in pdf_reader.pages:
#             text+=page.extract_text()
#             return text
# path=r'/kaggle/input/food-safety-manual/fpc-manual.pdf'
# pdf_text=extract_text_from_pdf(path)

In [7]:
def extract_text_from_pdf(path):
    text=""
    pdf_doc=fitz.open(path)
    for page in pdf_doc:
        text+=page.get_text()
    pdf_doc.close()
    return text
path=r'/kaggle/input/food-safety-manual/fpc-manual.pdf'
pdf_text=extract_text_from_pdf(path)

In [8]:
def clean_text(text):
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces/newlines
    text = re.sub(r'[^a-zA-Z0-9\s,.]', '', text)  # Remove special characters
    return text.strip()

In [9]:
cleaned_text=clean_text(pdf_text)
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)
chunks=text_splitter.split_text(cleaned_text)

In [10]:
from sentence_transformers import SentenceTransformer
embedding_model=SentenceTransformer('all-MiniLm-L6-v2')
chunk_embeddings=embedding_model.encode(chunks)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
dimension=len(chunk_embeddings[0])
index=faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))

In [12]:
def retrieve_similar_chunks(query, k=5):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [chunks[i] for i in indices[0]]

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
 
model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")
 


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [14]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [15]:
import time

In [18]:
def generate_answer(query):
    start_time=time.time()
    retrieved_chunks = retrieve_similar_chunks(query)
    context = " ".join(retrieved_chunks)
    prompt = f"context:{context}\n\nQuestion: {query}\n\nAnswer:"
    
    inputs = tokenizer(prompt, return_tensors="pt",padding=True, truncation=True).to("cuda")
    outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=200,temperature=0.7,do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    end_time = time.time()
    print(end_time-start_time)
    return response

In [20]:
query='how to make margarita cheese pizza?'
print(generate_answer(query))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


12.539113998413086
context:salad ingredients should be prepared in, and mixed with clean, sanitized equipment san itary type masher, sanitary mixing bowl, stainless steel longhandled spoon or fork, mechanical tumbler type mixer, etc.. The mixing opera tion is to be completed as quickly as possible and the finished salad immediately served or refrigerated. Additional Instructions for Specific Salads  Shrimp and Lobster Salad Immerse shrimp or lobster meat in boiling water for 30 seconds and then chill to 41F or less before preparing the salad. Fast chilling can be accomplished by 33 fo d P R O T E C T I O N T R A I N I N G M A N U A L N E W Y O R K C I T Y D E P A R T M E N T O F H E A L T H  M E N T A L H Y G I E N E 34 N E W Y O R K C I T Y D E P A R T M E N T O F H E A L T H  M E N T A L H Y G I E N E placing the meat in shallow pans in the freezer or refrigerator or on top of cracked ice.  Egg SaladAfter removing shells, use a hand strainer or colander to immerse hardboiled eggs in 